In [ ]:
import pandas as pd
import sys
import numpy as np
from datetime import datetime, timedelta
sys.path.append("../helper_functions")
import defillama_utils as dfl
import duneapi_utils as du
import opstack_metadata_utils as ops
sys.path.pop()

current_utc_date = datetime.utcnow().replace(hour=0, minute=0, second=0, microsecond=0)
print(current_utc_date)

In [ ]:
chains = [
    #L2s
     ['Optimism', 'L2']
    ,['Base', 'L2']
    ,['Arbitrum', 'L2']
    #non-dune L2s
    ,['ZkSync Era', 'L2']
    ,['Polygon zkEVM', 'L2']
    ,['Starknet', 'L2']
    ,['Linea', 'L2']
    ,['Mantle', 'L2']
    ,['Scroll', 'L2']
    ,['Boba', 'L2']
    ,['Metis', 'L2']
    ,['opBNB', 'L2']
    ,['Rollux', 'L2']
    ,['Manta', 'L2']
    ,['Kroma','L2']
    ,['Arbitrum%20Nova','L2']
    #L1
    ,['Ethereum', 'L1']
    #Others
    ,['Fantom', 'L1']
    ,['Avalanche', 'L1']
    ,['Gnosis' , 'L1']
    ,['Celo', 'L1']
    ,['Polygon', 'L1']
    ,['BSC', 'L1']
]

protocols = [
         ['aevo', 'L2',['optimism','arbitrum','ethereum']]
        ,['dydx', 'L2',['ethereum']]
        ,['immutablex', 'L2',['ethereum']]
        ,['apex-protocol', 'L2',['ethereum']]
        ,['brine.fi', 'L2',['ethereum']]
        ,['loopring', 'L2',['ethereum']]
        ,['aztec', 'L2',['ethereum']]
    ]

In [ ]:
p_agg = []
for p in protocols:
        d = dfl.get_single_tvl(p[0],p[2])
        d['chain_prot'] = p[0].title()
        d['layer'] = p[1]
        d['defillama_slug'] = 'protocols/' + p[0]
        d['source'] = 'protocol'
        # d['prot_chain'] = c
        p_agg.append(d)
df = pd.concat(p_agg)

df_sum = df.groupby(['date','chain_prot','layer','defillama_slug','source']).sum(['usd_value'])
df_sum.reset_index(inplace=True)

df_sum = df_sum[['date','usd_value','chain_prot','layer','defillama_slug','source']]
df_sum = df_sum.rename(columns={'chain_prot':'chain','usd_value':'tvl'})

print(df_sum.tail(5))

In [ ]:
c_agg = []
for c in chains:
        d = dfl.get_historical_chain_tvl(c[0])
        d['chain'] = c[0]
        d['layer'] = c[1]
        d['defillama_slug'] = c[0]
        d['source'] = 'chain'
        c_agg.append(d)

df_ch = pd.concat(c_agg)
df = pd.concat([df_ch,df_sum])
# Rename
df['chain'] = df['chain'].str.replace('%20', ' ', regex=False)
df['chain'] = df['chain'].str.replace('-', ' ', regex=False)
df.loc[df['chain'] == 'Optimism', 'chain'] = 'OP Mainnet'
df.loc[df['chain'] == 'BSC', 'chain'] = 'BNB'
df.loc[df['chain'] == 'Brine.Fi', 'chain'] = 'Brine'
df.loc[df['chain'] == 'Immutablex', 'chain'] = 'ImmutableX'


df = df[df['date'] <= current_utc_date ] #rm dupes at current datetime

df['date'] = pd.to_datetime(df['date']) - timedelta(days=1) #map to the prior day, since dfl adds an extra day

In [ ]:
# Add Metadata
chain_meta = pd.read_csv('../op_chains_tracking/inputs/chain_metadata.csv')
chain_meta = chain_meta.applymap(lambda x: x.strip() if isinstance(x, str) else x)

meta_cols = ['defillama_slug', 'is_op_chain','mainnet_chain_id','op_based_version']

df = df.merge(chain_meta[meta_cols], on='defillama_slug', how = 'left')

In [ ]:
df = ops.generate_alignment_column(df)

In [ ]:
# #Fill 
# df = df.fillna('')

In [ ]:
print(df.sample(10))
# print(df[df['chain'] == 'Boba'].tail(10))

print(df['chain'].unique())

In [ ]:
# Write to Dune
du.write_dune_api_from_pandas(df, 'dfl_chain_tvl',\
                             'TVL for select chains from DefiLlama')